In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_profiling as pp
import scipy.stats as stats
from sklearn import metrics
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt


%matplotlib inline

In [2]:
#I am going to try rerun model one with Ridge and Lasso

In [3]:
pd.set_option('display.max_columns', 999)

In [4]:
#imported both data sets

In [5]:
train = pd.read_csv('../datasets/train_clean.csv')

In [6]:
test = pd.read_csv('../datasets/test_clean.csv')

In [7]:
features=['total_bsmt_sf','gr_liv_area', 'tot_bath', 'tot_outdoor','house_age','lot_area',
            'garage_cars']
X_train = train[features]
X_test = test[features]
y_train_overfit = train['saleprice']

In [8]:
#Create PolynomialFeatures

In [9]:
poly= PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)

#instatiate a model

In [10]:
X_train_overfit = poly.fit_transform(X_train)
X_test_overfit = poly.fit_transform(X_test)

In [11]:
poly.get_feature_names(X_train.columns)

['total_bsmt_sf',
 'gr_liv_area',
 'tot_bath',
 'tot_outdoor',
 'house_age',
 'lot_area',
 'garage_cars',
 'total_bsmt_sf^2',
 'total_bsmt_sf gr_liv_area',
 'total_bsmt_sf tot_bath',
 'total_bsmt_sf tot_outdoor',
 'total_bsmt_sf house_age',
 'total_bsmt_sf lot_area',
 'total_bsmt_sf garage_cars',
 'gr_liv_area^2',
 'gr_liv_area tot_bath',
 'gr_liv_area tot_outdoor',
 'gr_liv_area house_age',
 'gr_liv_area lot_area',
 'gr_liv_area garage_cars',
 'tot_bath^2',
 'tot_bath tot_outdoor',
 'tot_bath house_age',
 'tot_bath lot_area',
 'tot_bath garage_cars',
 'tot_outdoor^2',
 'tot_outdoor house_age',
 'tot_outdoor lot_area',
 'tot_outdoor garage_cars',
 'house_age^2',
 'house_age lot_area',
 'house_age garage_cars',
 'lot_area^2',
 'lot_area garage_cars',
 'garage_cars^2']

In [12]:
X_train_overfit.shape

(2051, 35)

In [13]:
X_test_overfit.shape

(879, 35)

In [14]:
ss_overfit = StandardScaler()
Z_train_overfit = pd.DataFrame(ss_overfit.fit_transform(X_train_overfit))
Z_test_overfit = pd.DataFrame(ss_overfit.fit_transform(X_test_overfit))

In [15]:
Z_train_overfit.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
0,-0.739158,-0.040634,0.344941,-0.868739,-0.068429,0.512071,0.293112,-0.564043,-0.439481,-0.395602,-0.689560,-0.282416,-0.115896,-0.387832,-0.163627,0.056623,-0.672388,0.002604,0.219292,0.058356,0.164281,-0.688070,0.321179,0.487629,0.264089,-0.531407,-0.503972,-0.462755,-0.694342,-0.364565,0.256466,0.328936,0.053146,0.461262,0.095721
1,-0.321193,1.244529,1.583949,-0.681330,-0.763708,0.211664,0.293112,-0.345544,0.178278,0.400389,-0.554987,-0.738466,-0.068279,-0.135069,1.055089,1.684193,-0.416158,-0.460754,0.473134,0.741387,1.650782,-0.383341,-0.375087,0.787403,0.969817,-0.498520,-0.562666,-0.382883,-0.538918,-0.704962,-0.507416,-0.507972,-0.021697,0.229282,0.095721
2,-0.001049,-0.884084,-0.274563,-0.818763,0.693068,-0.317944,-1.013786,-0.144262,-0.407366,-0.222072,-0.602526,0.987825,-0.213987,-0.652023,-0.726915,-0.634424,-0.700565,0.205971,-0.451826,-0.951318,-0.393157,-0.700341,0.832361,-0.349214,-0.794503,-0.524272,-0.342618,-0.519865,-0.787596,0.357269,0.236705,0.109746,-0.124123,-0.633453,-1.003107
3,-1.497275,-0.110588,0.344941,-0.518908,-1.061685,-0.039047,0.293112,-0.832370,-0.809216,-0.886262,-0.665031,-1.107168,-0.531007,-0.846301,-0.217475,0.018437,-0.451346,-0.905945,-0.117914,0.021178,0.164281,-0.401747,-1.000844,0.054603,0.264089,-0.456492,-0.624438,-0.342335,-0.404218,-0.757728,-0.778775,-0.866647,-0.074880,0.035679,0.095721
4,-0.848096,-0.108589,-0.274563,-0.775034,2.447820,0.618586,0.293112,-0.612753,-0.507643,-0.660644,-0.659431,1.491333,-0.128067,-0.453712,-0.215954,-0.295774,-0.616158,2.220681,0.252677,0.022240,-0.393157,-0.671709,2.700820,0.239470,-0.088775,-0.517054,0.044939,-0.386177,-0.616630,3.409804,2.980869,3.357746,0.082594,0.543514,0.095721


In [16]:
lr=LinearRegression()

In [17]:
lr.fit(X_train_overfit, y_train_overfit)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [18]:
predictions = lr.predict(X_train_overfit)

In [19]:
r2_score(y_train_overfit, predictions)

0.8353993514418915

In [20]:
#create Ridge Model

In [21]:
ridge_mod = Ridge(alpha=10)

In [22]:
ridge_mod.fit(X_train_overfit, y_train_overfit)

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=2.70104e-20): result may not be accurate.
  overwrite_a=True).T


Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [23]:
ridge_predictions = ridge_mod.predict(X_train_overfit)

In [24]:
r2_score(y_train_overfit, ridge_predictions)

0.8353497314712952

In [25]:
r_alph = np.logspace(0, 10, 200)

In [26]:
ridge_mod = RidgeCV(alphas=r_alph, scoring='r2', cv=5)

In [27]:
ridge_mod = ridge_mod.fit(X_train_overfit, y_train_overfit)

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.67504e-20): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.25999e-19): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.68094e-20): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.06074e-20): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.72759e-20): result may not be accurate.
  overwrite_a=Tru

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.35866e-19): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.81804e-20): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.18721e-20): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.86679e-20): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.83358e-20): result may not be accurate.
  overwrite_a=Tru

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=2.0248e-19): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=2.74237e-20): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=2.04456e-20): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=2.80567e-20): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=2.83121e-20): result may not be accurate.
  overwrite_a=True

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=5.85624e-20): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=6.07549e-20): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=4.56669e-19): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=6.26365e-20): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=5.3423e-20): result may not be accurate.
  overwrite_a=True

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.3827e-19): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.45787e-19): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.09694e-18): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.51275e-19): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.37219e-19): result may not be accurate.
  overwrite_a=True

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=3.54015e-19): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=2.66709e-18): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=3.68275e-19): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=3.44362e-19): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=3.7317e-19): result may not be accurate.
  overwrite_a=True

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.04958e-18): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=9.96922e-19): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.06249e-18): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.13264e-18): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=8.57869e-18): result may not be accurate.
  overwrite_a=Tru

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=3.68062e-18): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=3.86652e-18): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=4.13036e-18): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=3.13093e-17): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=4.30563e-18): result may not be accurate.
  overwrite_a=Tru

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.4989e-17): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.56168e-17): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.50601e-17): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.57662e-17): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=1.68473e-17): result may not be accurate.
  overwrite_a=True

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=6.28655e-17): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=6.04632e-17): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=6.31636e-17): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=6.76704e-17): result may not be accurate.
  overwrite_a=True).T
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=7.0593e-17): result may not be accurate.
  overwrite_a=True

In [28]:
ridge_mod.alpha_

114.89510001873097

In [29]:
y_train_ridge_pred = ridge_mod.predict(X_train_overfit)
y_test_ridge_pred = ridge_mod.predict(X_test_overfit)

In [30]:
r2_score(y_train_overfit, y_train_ridge_pred)

0.8349502062476177

In [31]:
#create Lasso model

In [32]:
las_alph = np.logspace(-3, 0, 100)

In [33]:
las_mod = LassoCV(alphas=las_alph, cv=5)

In [34]:
las_mod = las_mod.fit(X_train_overfit, y_train_overfit)

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarni

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarni

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarni

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarni

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarni

...

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarni

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarni

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarni

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarni

/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/jerelnovick/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarni

In [35]:
# Generate predictions
y_train_las_pred = las_mod.predict(X_train_overfit)
y_test_las_pred = las_mod.predict(X_test_overfit)

# Evaluate model.
print(r2_score(y_train_overfit, y_train_las_pred))

0.8353991763458531


In [36]:
las_mod.alpha_

0.8697490026177834

In [37]:
r2_score(y_train_overfit, y_train_ridge_pred)

0.8349502062476177

In [38]:
ridge_mod.coef_

array([ 6.10402070e+01,  2.89081941e+01,  1.49637769e+03,  8.52585476e+00,
        6.13264070e+02,  2.42003966e+00, -2.50492063e+03, -2.32108295e-02,
       -2.61271384e-02,  2.61413795e+01,  4.97053485e-03, -7.10523279e-01,
        2.65016220e-04,  1.80866036e+01,  7.98607645e-06,  1.41940076e+01,
        1.20534108e-04,  8.26305211e-02, -5.86474918e-04,  1.50845658e+01,
       -5.27070998e+03, -8.40071263e+00, -2.37344847e+02,  1.35006510e-01,
       -5.43368975e+03, -2.88025494e-02,  1.25443430e-01, -8.55695306e-04,
        3.13232806e+01,  1.57672788e+00, -1.44202061e-02, -1.57224654e+02,
       -4.27937349e-06, -1.32790076e-02, -3.84785329e+03])

In [39]:
las_mod.coef_

array([ 5.93072659e+01,  1.99767741e+01,  1.38499843e+04,  1.03268083e+01,
        6.70867635e+02,  2.54662323e+00, -9.38025696e+03, -2.27357874e-02,
       -2.56439861e-02,  2.40408524e+01,  3.78852497e-03, -6.92633124e-01,
        2.97696074e-04,  2.04031296e+01, -1.38885733e-03,  1.68847389e+01,
       -2.32159439e-03,  1.46024461e-01, -5.63527094e-04,  1.74987422e+01,
       -6.84699573e+03, -6.77767902e+00, -3.20048773e+02,  6.53765566e-02,
       -7.00135649e+03, -2.86203618e-02,  1.13119204e-01, -8.60174714e-04,
        3.12030011e+01,  1.25533867e+00, -1.55407000e-02, -1.25303025e+02,
       -4.33069498e-06, -5.43463406e-03, -3.15432917e+03])

In [40]:
#created a new dataframe for me submission 

In [41]:
submission = pd.DataFrame(columns=[])

In [42]:
#assigned data to dataframe

In [43]:
#submission = pd.DataFrame(columns=[test['id']
submission['Id'] = test['id']
submission['SalePrice'] = y_test_ridge_pred

In [48]:
y_test_las_pred

array([144930.43882302, 216101.77965955, 189413.63654691, 111271.15389401,
       178346.32751544, 104774.42840743, 111732.43480643, 143821.5465799 ,
       188405.86051121, 182342.26137456, 174343.50559769, 110837.43289251,
       137458.73215096, 319402.59684306, 137105.48897483, 110969.75233782,
       140801.52389816, 125387.38160054, 212996.63980796, 138434.39685174,
       136571.25255414, 121058.82930908, 236931.96707592, 157716.9922186 ,
       185335.5377054 , 105844.73140812, 141875.85742716, 152444.13466673,
       129449.02436325,  81234.59623268, 105820.77299444, 119388.58129074,
       260731.41100829, 164378.38615607, 224795.9659325 , 170769.86664335,
       112207.52677161,  82952.88641673,  91380.3632115 , 207447.47402416,
       145161.57963547, 219000.82015211, 142331.46968195, 143912.94268359,
       217422.18368371, 108034.44076701, 220222.58795126, 124316.04938759,
       115545.98278046, 130284.98937155, 117906.06555349, 155741.79594063,
       263964.96402939, 1

In [45]:
submission.head()

,Id,SalePrice
0,2658,145263.856687
1,2718,215618.796793
2,2414,187633.640434
3,1989,110324.683171
4,625,178384.138377


In [46]:
submission.to_csv('../submissions/submission_six.csv',index=False)

In [49]:
submission['SalePrice'] = y_test_las_pred

In [50]:
submission.head()

,Id,SalePrice
0,2658,144930.438823
1,2718,216101.779660
2,2414,189413.636547
3,1989,111271.153894
4,625,178346.327515


In [51]:
submission.to_csv('../submissions/submission_seven.csv',index=False)